# Init

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append('../../../deconv_py/')
sys.path.append('../../infras/cellMix/')
sys.path.append('../../infras/cytof_data/')
sys.path.append('../../infras/ctpnet/')
sys.path.append('../../infras/')
sys.path.append('../../infras/dashboards/')
sys.path.append('../../experiments/')
sys.path.append('../../experiments/pipeline/')
sys.path.append('../../preprocess/intra_variance/')
sys.path.append('../../models/cell_proportions/')
sys.path.append('../../measures/cell_proportions_measures/')
sys.path.append('../../preprocess/cell_specifics/')
sys.path.append('../../preprocess/data_sets/')


In [3]:
from data_factory import DataFactory
from global_utils import GlobalUtils
from cytof_cell_count_infra import CytofCellCountInfra
from cell_proportions_experiments import  CellProportionsExperiments
import exploration_cytof_plots as cytof_plots

from pp_entropy_based import PpEntropyBased
from pp_dep_de_based import  PpDepDeBased
from cell_proportions_measure import CellProportionsMeasure
from pp_clean_high_intra_var import PpCleanHighIntraVar
from pp_clean_irrelevant_proteins import PpCleanIrrelevantProteins
from pp_empty import PpEmpty
from pp_entropy_based_only_largest import PpEntropyBasedOnlyLargest
from aggregate_intra_variance import AggregateIntraVariance
from pipeline_deconv import PipelineDeconv
from deconv_py.preprocess.base import BasePreprocess as PP_base
from deconv_py.preprocess.cell_specific import CellSpecific as PP_proteins
from deconv_py.preprocess.cell_specifics.pp_svm_signature import PpSvmSignature
from deconv_py.preprocess.cell_specifics.pp_entropy_based_totel_sum import PpEntropyBasedTotelSum
from deconv_py.preprocess.cell_specifics.pp_floor_under_quantile import PpFloorUnderQuantile
from pick_data_set import PickDataSet
# from deconvolution_results_plots import DeconvolutionResultsPlots

from basic import BasicDeconv
from regression import RegressionDeconv
from generalized_estimating_equations import GeneralizedEstimatingEquations
from robust_linear_model import RobustLinearModel
from deconvolution_model import DeconvolutionModel
from sklearn import linear_model


from pp_keep_specific_cells  import  PpKeepSpecificCells
from pp_agg_to_specific_cells import PpAggToSpecificCells

# from deconv_py.infras.data_factory import DataFactory
from deconv_py.infras.data_loader import DataLoader
from deconv_py.models.base import Base as Models_base
from deconv_py.models.cell_proportions_models import CellProportions
from deconv_py.models.cell_specific_models import CellSpecificPerPermutation
from deconv_py.experiments.cell_specific import CellSpecificMetricsPlot
from cellMix_coordinator import CellMixCoordinator

from infras.ctpnet.ctpnet_coordinator import CtpNetCoordinator


import pandas as pd
import numpy as np
from functools import partial
import multiprocessing
from sklearn import pipeline
import itertools
from scipy.optimize import least_squares
from sklearn.metrics import mean_squared_error
from functools import partial
from scipy.optimize import minimize
import scipy.optimize
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
import os
import pickle as pkl 
from sklearn.decomposition import PCA
from IPython.display import display, HTML
import statsmodels.api as sm
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform
import mygene



%connect_info

{
  "shell_port": 63779,
  "iopub_port": 63780,
  "stdin_port": 63781,
  "control_port": 63782,
  "hb_port": 63783,
  "ip": "127.0.0.1",
  "key": "c323d819-e0950ecae138b388754880b8",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-7dada7ac-85f1-488b-bf0f-768377285680.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


# build data

## signature

In [4]:
data_factory = DataFactory()
A_all_vs,B_all_vs = data_factory.load_IBD_all_vs("Intensity",index_func=lambda x:x.split(";")[0],log2_transformation=True)


spec_cells = PpKeepSpecificCells()
agg_iv,pp_irl_prot  = AggregateIntraVariance(),PpCleanIrrelevantProteins()
pp_entropy = PpEntropyBased()

steps_all_cells = [("kepp_specific_cells",spec_cells),
                  ("AggregateIntraVariance",agg_iv),("cleen_irrelevant_proteins",pp_irl_prot),
                  ("PpEntropyBased",pp_entropy)]

_params = {"AggregateIntraVariance__how":"median","PpEntropyBased__n_genes_per_cell":500,"PpEntropyBased__gene_entropy_trh":1,
          "PpEntropyBased__with_norm":True,"PpEntropyBased__number_of_bins":0}


pip_all_cells = pipeline.Pipeline(steps=steps_all_cells)
pip_all_cells.set_params(**_params)
A_sig,B_sig = pip_all_cells.transform([A_all_vs,B_all_vs])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (5,6,2764,2765,2773,2774,2775,2776) have mixed types. Specify dtype option on import or set low_memory=False.
  
C:\Users\Shenorr\AppData\Roaming\Python\Python36\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [5]:
tmp = A_all_vs.copy(deep=True)
tmp = tmp.rename(columns={col:col.split('_0')[0] for col in tmp.columns})
A_all_vs = tmp.T.groupby(level=0).median().T

In [6]:
A_sig.head().columns

Index(['Intensity NOT_BCellmemory', 'Intensity NOT_BCellnaive',
       'Intensity NOT_BCellplasma', 'Intensity NOT_CD4TCellTcm',
       'Intensity NOT_CD4TCellTem', 'Intensity NOT_CD4TCellTemra',
       'Intensity NOT_CD4TCellnTregs', 'Intensity NOT_CD4TCellnaive',
       'Intensity NOT_CD8TCellTem', 'Intensity NOT_CD8TCellTemra',
       'Intensity NOT_CD8TCellnaive', 'Intensity NOT_DendriticCD1c',
       'Intensity NOT_DendriticCD304', 'Intensity NOT_Monocytesclassical',
       'Intensity NOT_Monocytesintermediate',
       'Intensity NOT_Monocytesnonclassical',
       'Intensity NOT_NKCellsCD56bright', 'Intensity NOT_NKCellsCD56dim'],
      dtype='object', name='cell')

## no em model

In [7]:
rlm =RobustLinearModel(weight_sp = False)
cell_abundance_over_samples_df = rlm._run_deconvolution_over_samples(B_sig, A_sig, None, None)
# cell_abundance_over_samples_df = (cell_abundance_over_samples_df / cell_abundance_over_samples_df.sum(
#             axis=0)).round(2)
cell_abundance_over_samples_df


,24_v1,24_v2,24_v3,26_v1,26_v2,26_v3,27_v1,27_v2,27_v3,28_v1,...,38_v2,38_v3,39_v1,39_v2,39_v3_190722234036,40_v1,40_v3,42_v1,42_v2,42_v3
cell,,,,,,,,,,,,,,,,,,,,,
Intensity NOT_BCellmemory,0.000365,0.000269,0.000456,0.000543,0.000442,4.421758e-04,0.000335,0.000376,0.000357,0.000283,...,0.000248,0.000367,0.000376,0.000353,0.000383,0.000535,0.000235,0.000302,0.000325,0.000327
Intensity NOT_BCellnaive,0.000179,0.000309,0.000212,0.000000,0.000126,1.326963e-04,0.000225,0.000095,0.000108,0.000189,...,0.000459,0.000104,0.000105,0.000228,0.000145,0.000172,0.000351,0.000174,0.000126,0.000132
Intensity NOT_BCellplasma,0.000476,0.000292,0.000641,0.000438,0.000500,5.627864e-04,0.000352,0.000382,0.000318,0.000387,...,0.000471,0.000420,0.000352,0.000335,0.000344,0.000703,0.000350,0.000295,0.000260,0.000335
Intensity NOT_CD4TCellTcm,0.000000,0.000000,0.000113,0.000000,0.000000,1.416771e-05,0.000024,0.000067,0.000000,0.000000,...,0.000080,0.000000,0.000000,0.000000,0.000023,0.000000,0.000000,0.000000,0.000000,0.000000
Intensity NOT_CD4TCellTem,0.000265,0.000281,0.000222,0.000263,0.000329,2.728649e-04,0.000180,0.000113,0.000127,0.000277,...,0.000253,0.000341,0.000275,0.000232,0.000172,0.000354,0.000264,0.000227,0.000244,0.000263
Intensity NOT_CD4TCellTemra,0.000168,0.000163,0.000279,0.000173,0.000214,3.737202e-04,0.000139,0.000176,0.000185,0.000123,...,0.000054,0.000106,0.000122,0.000105,0.000098,0.000308,0.000151,0.000055,0.000089,0.000127
Intensity NOT_CD4TCellnTregs,0.000000,0.000000,0.000044,0.000000,0.000001,0.000000e+00,0.000010,0.000000,0.000000,0.000049,...,0.000157,0.000063,0.000000,0.000000,0.000038,0.000000,0.000000,0.000011,0.000004,0.000009
Intensity NOT_CD4TCellnaive,0.000006,0.000089,0.000000,0.000069,0.000136,0.000000e+00,0.000141,0.000074,0.000178,0.000076,...,0.000000,0.000000,0.000111,0.000001,0.000108,0.000159,0.000239,0.000018,0.000116,0.000202
Intensity NOT_CD8TCellTem,0.000268,0.000230,0.000199,0.000284,0.000295,3.033828e-04,0.000176,0.000202,0.000254,0.000245,...,0.000163,0.000156,0.000247,0.000193,0.000275,0.000286,0.000362,0.000328,0.000216,0.000207


## no_em_model_results

In [8]:
def return_measure(deconv_result,return_mean = True,method = "pearson" ) : 
    ccci = CytofCellCountInfra(cluster_info_path= r"C:\Repos\deconv_py\deconv_py\infras\cytof_data\raw_data\CyTOF.features.and.clusters.info.xlsx",
                     cytof_data_path=r"C:\Repos\deconv_py\deconv_py\infras\cytof_data\raw_data\filtered.esetALL.CyTOF.abundance.only.xlsx")
    cpm = CellProportionsMeasure()
    deconv_result,known_results = ccci.return_mass_and_cytof_not_none_cells_counts(deconv_result,filter_by_version="")


    if not any([(col in known_results.index.tolist()) for col in deconv_result.index]) :
        columns_mapping = GlobalUtils.get_corospanding_cell_map_from_lists(known_results.index.to_list(),
                                                                               deconv_result.index.to_list())
        known_results = known_results.rename(index=columns_mapping)

    mixtures_map = GlobalUtils.get_corospanding_mixtures_map_from_lists(deconv_result.columns,known_results.columns)
    deconv_result = deconv_result.rename(columns=mixtures_map)


    measure_function = lambda x,y : x.corrwith(y,method=method, axis=0)
    res = measure_function(deconv_result,known_results)
    
    if return_mean : 
        return res.mean()
    return res

return_measure(cell_abundance_over_samples_df)

0.19885964175220885

## build Ransac

###  load exs data

In [10]:
# def return_ens_id_to_gene_name(list_of_ids):
#     mg = mygene.MyGeneInfo()
#     ginfo = mg.querymany(list_of_ids, scopes='ensembl.gene')

#     ensg_id_to_gene_name_mapping = {}
#     for g in ginfo:
#         if ("query" in g.keys()) and ("symbol" in g.keys()):
#             ensg_id_to_gene_name_mapping[g["query"]] = g["symbol"]
#     return ensg_id_to_gene_name_mapping

# accs_ratio = pd.read_excel('../../data/protein_to_gene_ratio.xlsx')
# mapping = return_ens_id_to_gene_name(accs_ratio.Accessions.tolist())
# accs_ratio['Accessions'] = accs_ratio['Accessions'].map(mapping)
# accs_ratio = accs_ratio.drop(columns=['Unnamed: 2']).set_index('Accessions')

# accs_ratio = accs_ratio.reset_index().dropna().set_index('Accessions')
accs_ratio = pd.read_excel(r'check_protein_to_mrna_hypos/protein_to_gene_ratio_after_mapping.xlsx')

### build mrna data

In [11]:
MASS_CELL_NAMES_TO_HAMRNA_CELL_MAPPING =  {'BCellmemory': 'memory B-cell',
                                    'BCellnaive' : 'naive B-cell',
                                    'BCellplasma' : None,
                                    'CD4TCellmTregs' : None,
                                    'CD4TCellnaive':'naive CD4 T-cell',
                                    'CD4TCellnTregs':None,
                                    'CD4TCellTcm' : 'memory CD4 T-cell' ,
                                    'CD4TCellTem' : None,
                                    'CD4TCellTemra' :None,
                                    'CD4TCellTh1':None,
                                    'CD4TCellTh17':None,
                                    'CD4TCellTh2':None,
                                    'CD8TCellnaive':'naive CD8 T-cell',
                                    'CD8TCellTcm' : 'memory CD8 T-cell',
                                    'CD8TCellTem' : None ,
                                    'CD8TCellTemra' : None,
                                    'DendriticCD1c' : None,
                                    'DendriticCD304' : None,
                                    'Erythrocytes' : None,
                                    'Monocytesclassical': 'classical monocyte' ,
                                    'Monocytesintermediate' : 'intermediate monocyte',
                                    'Monocytesnonclassical': 'non-classical monocyte',
                                    'NKCellsCD56bright' : 'NK-cell',
                                    'NKCellsCD56dim': 'NK-cell',
                                    'Thrombocytes' : None,
                                    'Granulocyteseosinophils' : "eosinophil",
                                    'GranulocytesBasophil' : "basophil"}

rna_blood_cell_df = pd.read_csv("../../data/gene_expression/human atlas/rna_blood_cell.tsv",sep="\t")
rna_blood_cell_df =rna_blood_cell_df.loc[~rna_blood_cell_df[["Gene name","Blood cell"]].duplicated()]
ptmp_rna_cell_df = rna_blood_cell_df.pivot(index = "Gene name",columns="Blood cell",values="pTPM")
gene_df = rna_blood_cell_df.pivot(index = "Gene name",columns="Blood cell",values="NX")


In [12]:
cells_list = A_sig.columns
cells_name_mapping = MASS_CELL_NAMES_TO_HAMRNA_CELL_MAPPING

run_time_cells_to_Mass_cells_mapping = GlobalUtils.get_corospanding_cell_map_from_lists(cells_list,cells_name_mapping.keys())
ctpnet_to_run_time_cells_mapping = {cells_name_mapping[run_time_cells_to_Mass_cells_mapping[run_time_cell]] : run_time_cell for run_time_cell in cells_list }


In [13]:
gene_df_sig_cells = gene_df[[c for c in ctpnet_to_run_time_cells_mapping.keys() if c is not None]].rename(columns = ctpnet_to_run_time_cells_mapping)

### compare signatures as test

In [14]:
_mutual_genes = gene_df_sig_cells.index.intersection(accs_ratio.index)

gene_df_sig_cells_mutual = gene_df_sig_cells.loc[_mutual_genes].copy(deep=True)
accs_ratio_mutual_genes = accs_ratio.loc[_mutual_genes]

prot_from_gene_infer = np.power(gene_df_sig_cells_mutual,10)*np.power(accs_ratio_mutual_genes.values,10)

ValueError: Unable to coerce to DataFrame, shape must be (0, 9): given (0, 2)

In [ ]:
A_for_compr = A_all_vs.copy(deep=True)
A_for_compr.index = A_for_compr.index.droplevel(0)

_mutual_a_ref_genes = A_for_compr.index.intersection(prot_from_gene_infer.index)
_mutual_a_ref_cells = A_for_compr.columns.intersection(prot_from_gene_infer.columns)

A_for_compr = A_for_compr.loc[_mutual_a_ref_genes]
prot_from_gene_infer_for_comp = prot_from_gene_infer.loc[_mutual_a_ref_genes]

A_for_compr = A_for_compr[_mutual_a_ref_cells]
prot_from_gene_infer_for_comp = prot_from_gene_infer_for_comp[_mutual_a_ref_cells]

A_for_compr = A_for_compr[~A_for_compr.index.duplicated()]
prot_from_gene_infer_for_comp = prot_from_gene_infer_for_comp[~prot_from_gene_infer_for_comp.index.duplicated()]

In [ ]:
for cell in A_for_compr.columns : 
    _prot_sig =  A_for_compr[cell]
    _gene_sig = prot_from_gene_infer_for_comp[cell]
    
    _prot_sig = _prot_sig[(_prot_sig>_prot_sig.quantile(0.8)) &(_prot_sig<_prot_sig.quantile(0.9))]
    _gene_sig = _gene_sig[(_gene_sig>_gene_sig.quantile(0.8)) &(_gene_sig<_gene_sig.quantile(0.9))]
    
    _mu = _gene_sig.index.intersection(_prot_sig.index)
    
    _prot_sig = _prot_sig[_mu]
    _gene_sig = _gene_sig[_mu]
    
    plt.scatter(_gene_sig,_prot_sig)
    plt.show()

### build expected mixtures  from gene imputation

In [ ]:
expected_mixtures_from_gene_imputation = prot_from_gene_infer.dot(cell_abundance_over_samples_df.loc[prot_from_gene_infer.columns])

### plot

In [ ]:
expc_melted = expected_mixtures_from_gene_imputation.reset_index().melt(id_vars=['index'],value_vars=expected_mixtures_from_gene_imputation.columns)
mixture_melted = B_sig.reset_index(level=1).melt(id_vars=['Gene names'],value_vars=B_sig.columns)

In [ ]:
expc_melted = expc_melted.set_index(['index','variable'])
mixture_melted = mixture_melted.set_index(['Gene names','variable'])

In [ ]:
_mutual = expc_melted.index.intersection(mixture_melted.index)
mixture_melted =  mixture_melted.loc[_mutual]
expc_melted = expc_melted.loc[_mutual]

In [ ]:
mixture_melted = mixture_melted[~mixture_melted.index.duplicated()]
expc_melted = expc_melted[~expc_melted.index.duplicated()]

In [ ]:
# for mixture
plt.scatter(mixture_melted,expc_melted)

### Ransac

In [ ]:
X, y = expc_melted,mixture_melted

ransac = linear_model.RANSACRegressor()
ransac.fit(X, y)
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)
prop = ransac.estimator_.coef_

lw = 2
plt.scatter(X[inlier_mask], y[inlier_mask], color='yellowgreen', marker='.',
            label='Inliers')
plt.scatter(X[outlier_mask], y[outlier_mask], color='gold', marker='.',
            label='Outliers')

plt.legend(loc='lower right')
plt.xlabel("Input")
plt.ylabel("Response")
plt.show()

In [ ]:
B_sig.index

In [ ]:
ransac.fit(expected_mixtures_from_gene_imputation, B_sig)
